In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_name = "AI-Sweden-Models/gpt-sw3-126m"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model.eval()
model.to(device)

# Get number of parameters
num_params = sum(p.numel() for p in model.parameters())
num_params

/home/edgelab/Tim/miniconda3/envs/pytorch-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


135780864

In [2]:
prompt = "Jag är en AI och jag"
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

generated_token_ids = model.generate(
    inputs = input_ids,
    max_new_tokens = 100,
    do_sample=True,
    temperature = 0.6,
    top_p=1
)[0]

generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
print(generated_text)

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.

Jag är en AI och jag är en robot.


In [4]:
import json
import pandas as pd
path = "../../final/smaller_test.jsonl"

with open(path, "r") as file:
    data = [json.loads(line) for line in file] # jsonl file 

print(data[15]['text'][:-1])
print(data[15]['text'][-1])


[{'<human>': 'Vad var den stora lögnen angående det amerikanska presidentvalet 2020?'}, {'<bot>': 'Den stora lögnen avser det falska påståendet från tidigare president Donald Trump att valet stals genom omfattande valfusk. Det finns dock inga bevis som stöder detta påstående och valresultatet har certifierats som korrekt av statliga och federala myndigheter.'}, {'<human>': 'Varför var tron så utbredd?'}]
{'<bot>': 'För det första, år 2016 trodde Trump att han skulle förlora valet och började säga att Amerikas val var "riggade". Trump lyckades fortfarande vinna trots att han hade färre röster totalt. Men år 2020 fick Trump 10 miljoner fler röster totalt än år 2016, men han sades fortfarande förlora valet. \nDessutom, natten innan valet slutade, var rösterna till förmån för Trump. Men poströstsedlarna, som vanligtvis räknas på natten, var inte till Trumps fördel. Morgonen efter det var Trump inte längre i ledningen. \nPå en något annorlunda anmärkning är en annan möjlig orsak till detta 

In [11]:
import json
from datahandler import tokenize 

path = "100_examples.jsonl"

with open(path, "r") as file: 
    data = [json.loads(line) for line in file]

tokenize(data[24], '<s>', '<|endoftext|>')


<|endoftext|>
<s>User
Jan Frederik Veldkamp (31 March 1941, Amsterdam - 12 November 2017) was a Dutch botanist. The standard author abbreviation Veldkamp is used to indicate this person as the author when citing a botanical name.
<s>User
What is known about the author Jan Frederik Veldkamp?
<s>Bot
Jan Frederik Veldkamp Jan Frederik Veldkamp (31 March 1941, Amsterdam - 12 November 2017) was a Dutch Botanist.
<s>


tensor([[63423,    18,     3, 63423,    18,     2, 15088,    18, 30777, 15081,
           435,   825,  9980,   383, 63480, 63456,  7464, 63423, 63456, 63491,
         63489, 63456, 63446, 19171,   381, 63423, 63456, 63459,  7540, 63423,
         63459, 63455, 63456, 63499, 63462,   545,   268, 23730, 39040,   412,
         63443,   619,  3844,  4921, 62499, 13841,   435,   825,  9980,   428,
          2067,   341, 19440,   593,   854,   578,   306,  4921,  1206,  5888,
           291,   268, 39040,  1111,  1998, 63443,    18,     2, 15088,    18,
          4950,   428,  4191,   998,   306,  4921,  5526, 15081,   435,   825,
          9980, 63495,    18,     2, 22493,    18, 30777, 15081,   435,   825,
          9980,  5526, 15081,   435,   825,  9980,   383, 63480, 63456,  7464,
         63423, 63456, 63491, 63489, 63456, 63446, 19171,   381, 63423, 63456,
         63459,  7540, 63423, 63459, 63455, 63456, 63499, 63462,   545,   268,
         23730, 51498,   412, 63443,    18,     2]])

In [6]:
# from tok import tokenize_file, CHAT_TURN_FORMATS, ROLEMAP 

SPECIAL_TOKENS = tokenizer.special_tokens_map
EOS_TOKEN = SPECIAL_TOKENS["eos_token"]
BOS_TOKEN = SPECIAL_TOKENS["bos_token"]

SPECIAL_TOKENS

{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}

In [10]:
def prepare_data(samples : list):
    inputs = []
    targets = []
    for sample in samples: 
        context = sample['text'][:-1]
        response = sample['text'][-1]
        inputs.append(f"{BOS_TOKEN} {context} {EOS_TOKEN}")
        targets.append(f"{response} {EOS_TOKEN}")
    return inputs, targets

# Example usage
inputs, targets = prepare_data(data)

max_length = 48
input_encodings = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt", max_length=max_length, add_special_tokens=True)
target_encodings = tokenizer(targets, padding=True, truncation=True, return_tensors="pt", max_length=max_length, add_special_tokens=True)

# Make sure that we use Dataset class
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings.input_ids)
    
train_dataset = TextDataset(input_encodings, target_encodings.input_ids)
train_dataset.encodings["input_ids"]
len(train_dataset[0]["input_ids"])

split = 0.8
n = len(train_dataset)
train_size = int(split * n)
val_size = n - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])


train_dataset[0]



# TODO: Dataloader 
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)


/tmp/ipykernel_4237/103996050.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([    2, 63423, 45416, 63457, 63523, 32650, 63503, 12273,  1423, 29190,
         15167,   268, 13555, 10495,   346,   268,  4399,   564, 10621,  6461,
           268, 11877,   515,   268, 41010,   606, 12111,   623, 63446,   600,
          2587, 45884, 63446,   348,  4328,   504, 41549, 39652, 63423,     3,
             0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])}

In [ ]:
# Finetune
from transformers import Trainer, TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    # max_steps=500,
    fp16=True
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    dataset_text_field='text',
    max_seq_length=max_length
    )

trainer.train()

In [6]:
model.eval()

# Create a prompt
prompt = "<s>Vad är 4 plus 4?<|endoftext|>"
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

# Generate a response
generated_token_ids = model.generate(
    inputs = input_ids,
    max_new_tokens = 200,
    do_sample=True,
    temperature = 0.6,
    top_p=1
)[0]

generated_text = tokenizer.decode(generated_token_ids, skip_special_tokens=True)
print(generated_text)

Vad är 4 plus 4? 3.25*162.5\xa0m\xa0Vatten\xa0och\xa0vätska\xa0och\xa0elektrolyter\xa0(i)'}, {'<human>': 'Finns det några specifika säkerhetsriktlinjer att följa när man använder Pregabalin Krka?'}, {'<bot>': 'Pregabalin Krka, Följande försiktighetsåtgärder gäller: Spårbarhet För att underlätta spårbarhet av biologiska läkemedel ska läkemedlets namn och tillverkningssatsnummer dokumenteras., Pediatrisk population Användning av pregabalin till barn rekommenderas inte på grund av risken för överdosering. Pediatrisk population ska inte övervakas med avseende på säkerhet eller effekt hos barn under 18 år., Administrering av pregabalin till barn ska ske under medicinsk övervakning av läkare med erfarenhet av behandling av pediatrisk population., Interaktioner och kontraindikationer Interaktioner med andra mediciner har
